# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [177]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [178]:
train_df.shape

(6000, 61)

In [179]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [180]:
y.mean()

0.5

In [181]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [182]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
import numpy as np
from sklearn.metrics import mean_squared_error

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

### Árbol de decisión

In [183]:
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)
treereg.fit(X_train, y_train)
y_pred_a = treereg.predict(X_test)
metrics.f1_score(y_pred_a, y_test), metrics.accuracy_score(y_pred_a, y_test)

(0.5428002654280026, 0.5406666666666666)

### Regresión Logistica

In [184]:
logisticreg = LogisticRegression()
logisticreg.fit(X_train, y_train)
y_pred_l = logisticreg.predict(X_test)
metrics.f1_score(y_pred_l, y_test), metrics.accuracy_score(y_pred_l, y_test)

(0.6104529616724739, 0.6273333333333333)

# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

### 100 Arboles de decisión con max_depth=None

In [185]:
import pandas as pd
import numpy as np

In [186]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [187]:
n_estimators = 100
np.random.seed(123)
n_samples = X_train.shape[0]
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

In [188]:
np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

trees = {}
for i in range(n_estimators):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [189]:
y_pred_n = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_n.iloc[:, i] = trees[i].predict(X_test)

y_pred_n.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1782,1,1,1,1,0,1,1,1,1,1,...,1,1,0,1,0,0,0,1,1,0
3917,0,1,0,0,0,1,1,0,0,1,...,1,0,1,0,0,1,0,0,0,1
221,1,1,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,0,1,1
2135,0,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
5224,1,0,1,1,0,0,1,0,1,0,...,1,0,0,0,1,0,0,0,0,1


In [190]:
y_pred_n.sum(axis=1)[:10]

1782    62
3917    45
221     44
2135    22
5224    27
1168    40
879     18
156     25
1657    62
323     23
dtype: int64

In [191]:
y_pred_n = (y_pred_n.sum(axis=1) >= (n_estimators / 2)).astype(np.int)

from sklearn import metrics
metrics.f1_score(y_pred_n, y_test)

0.6453226613306652

In [192]:
metrics.accuracy_score(y_pred_n, y_test)

0.6419191919191919

In [193]:
np.sqrt(mean_squared_error(y_test, y_pred_n))

0.5983985361619863

### con scikit-learn - max_depth=None

In [276]:
bag_treereg1 = BaggingRegressor(DecisionTreeRegressor(max_depth=None, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

bag_treereg1.fit(X_train, y_train)
y_pred_tr1 = bag_treereg2.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred_tr1))

0.4737516353628809

### 100 Árboles de decisión con max_depth=2

In [194]:
n_estimators = 100
np.random.seed(123)
n_samples = X_train.shape[0]
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

In [195]:
np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

trees = {}
for i in range(n_estimators):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=2, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [196]:
y_pred_2 = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_2.iloc[:, i] = trees[i].predict(X_test)

y_pred_2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1782,1,1,0,1,1,0,1,1,1,1,...,0,1,1,0,1,0,1,0,1,0
3917,1,1,0,0,0,1,0,0,0,1,...,1,0,1,0,0,1,1,0,0,1
221,1,1,0,1,1,0,0,0,1,0,...,0,0,1,0,1,0,1,0,0,0
2135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
5224,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [197]:
y_pred_2.sum(axis=1)[:10]

1782    55
3917    62
221     33
2135     7
5224    17
1168    15
879      0
156      2
1657    75
323     18
dtype: int64

In [198]:
y_pred_2 = (y_pred_2.sum(axis=1) >= (n_estimators / 2)).astype(np.int)

from sklearn import metrics
metrics.f1_score(y_pred_2, y_test)

0.6498015873015873

In [199]:
metrics.accuracy_score(y_pred_2, y_test)

0.6434343434343435

In [200]:
np.sqrt(mean_squared_error(y_test, y_pred_2))

0.5971311887396743

### con scikit-learn - max_depth=2

In [275]:
bag_treereg2 = BaggingRegressor(DecisionTreeRegressor(max_depth=2, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

bag_treereg2.fit(X_train, y_train)
y_pred_tr2 = bag_treereg2.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred_tr2))

0.4737516353628809

### 100 Regresiones Logísticas

In [201]:
bag_reglog = BaggingRegressor(LogisticRegression(), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)
bag_reglog.fit(X_train, y_train)
y_pred_l2 = bag_reglog.predict(X_test)

In [202]:
np.sqrt(mean_squared_error(y_test, y_pred_l2))

0.5684924241919468

# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

### Predicción usando majority voting con Sklearn

In [205]:
from sklearn.ensemble import BaggingClassifier

In [206]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)
clf.fit(X_train, y_train)
y_pred_mv = clf.predict(X_test)

In [207]:
metrics.f1_score(y_pred_mv, y_test), metrics.accuracy_score(y_pred_mv, y_test)

(0.632225063938619, 0.6368686868686869)

# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [250]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.05).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5728643216080401

In [251]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.1).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5720000000000001

In [252]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.15).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5717161550075491

In [253]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.2).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5751765893037335

In [254]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.25).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5709954522486105

In [255]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5822532402791625

In [256]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.35).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5774225774225774

In [257]:
clf_pp = DecisionTreeClassifier()
clf_pp.fit(X_train, y_train)
y_pred_pp = (clf_pp.predict_proba(X_test)[:,1] >= 0.4).astype(bool)
metrics.f1_score(y_pred_pp, y_test)

0.5758637956935404

### El porcentaje que maximiza el F1_Score es 30% con un resultado de 0,582

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

### Predicción usando Weighted voting con Sklearn

In [140]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)
clf.fit(X_train, y_train)
y_pred_o = clf.predict(X_test)

In [142]:
metrics.f1_score(y_pred_o, y_test), metrics.accuracy_score(y_pred_o, y_test)

(0.632225063938619, 0.6368686868686869)

In [258]:
errors = np.zeros(clf.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], clf.n_estimators))

for i in range(clf.n_estimators):
    oob_sample = ~clf.estimators_samples_[i]
    y_pred_ = clf.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = clf.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
y_pred_o = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int)

In [259]:
metrics.f1_score(y_pred_o, y_test), metrics.accuracy_score(y_pred_o, y_test)

(0.6386213887480993, 0.6398989898989899)

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [228]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.1).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6759133964817321

In [229]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.15).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6805745554035567

In [230]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.2).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.685953293830603

In [231]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.25).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6891117478510028

In [232]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6956845238095237

In [233]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.35).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6997287872917473

In [234]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.4).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6939958592132505

In [235]:
y_pred_o1 = (clf.predict_proba(X_test)[:,1] >= 0.45).astype(bool)
metrics.f1_score(y_pred_o1, y_test)

0.6795332136445241

### La probabilidad del 35% maximiza el F1-Score

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [264]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.1).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6759133964817321

In [265]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.15).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6805745554035567

In [266]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.2).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.685953293830603

In [268]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.25).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6891117478510028

In [269]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6956845238095237

In [270]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.35).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6997287872917473

In [271]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.4).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6939958592132505

In [272]:
y_pred_l2 = (clf.predict_proba(X_test)[:,1] >= 0.45).astype(bool)
metrics.f1_score(y_pred_l2, y_test)

0.6795332136445241

### El 35% de probabilidad que maximiza la regresión logistica (l2) con 0,699